What I want to analyze here?
1. Visual profile of each timestep
2. Function-duration distribution

In [ ]:
# To read and parse outputted .json file
import json

# Main data frame
import pandas as pd

# Draw interactive graphs with plotly
import plotly.express as px
from plotly.offline import init_notebook_mode

# To use create directory and store progress
import os
from datetime import datetime, date

init_notebook_mode(connected=True)

In [ ]:
filename = '../fAOS_simulation_time_profile'

data = []

step_count = 0
max_load_step = 10

with open(filename + '.json') as file:
  for r in json.load(file)['record']:
    data.append({
      'name': r['name'],
      'start': datetime.fromtimestamp(r['start']),
      'end': datetime.fromtimestamp(r['start'] + r['dur'] / 1e6), # datetime.fromtimestamp(r['end']),
      'dur, μs': r['dur'],
      'tid': r['tid']
    })

    if r['name'] == 'one timestep':
      step_count += 1
      if not step_count < max_load_step: break

In [ ]:
def customize_figure(fig):
  for i, d in enumerate(fig['data']):
    if d['name'] in ['void Manager::initializes()', 'one timestep']:
      d.width = 0.1
      d['y'] = d['y'].astype('float64') - 0.6

    if d['name'] in ['void Particles::push()', 'void Fields::propagate()', 'void Manager::diagnose(size_t) const']:
      d.width = 0.2
      d['y'] = d['y'].astype('float64') - 0.6

    if d['name'] == 'field interpolation process':
      d.width = 0.2
      d['y'] = d['y'].astype('float64') - 0.35

    if d['name'] == 'particle push process':
      d.width = 0.2
      d['y'] = d['y'].astype('float64') - 0.125

    if d['name'] == 'current decomposition process':
      d.width = 0.2
      d['y'] = d['y'].astype('float64') + 0.125

    if d['name'] == 'adding particles via open boundaries':
      d.width = 0.2
      d['y'] = d['y'].astype('float64') + 0.35


In [ ]:
df = pd.DataFrame(data)

fig = px.timeline(
  df,
  x_start='start', x_end='end', y='tid',
  hover_data=['name', 'dur, μs'],
  color='name',
  height=800,
  width=1200,
)

customize_figure(fig)

fig.update_yaxes(autorange="reversed")
fig.update_xaxes(rangeslider_visible=True)

fig.show()

if not os.path.exists('log/'):
  os.mkdir('log/')

fig.write_image(date.today().strftime('log/%b-%d-%y.png'), width=1500)

In [ ]:
fig = px.histogram(
  df,
  x='name',
  y='dur, μs',
  color='name'
)

fig.show()